In [ ]:
import pandas as pd
import numpy as np
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain import hub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA

def retrieval(information):

    FAISS_DB_PATH = "faiss_KPIS"

    embedding_model = OllamaEmbeddings(model="llama3.2:3b")
    print("\n🔄 Loading existing FAISS index...")
    vector_db = FAISS.load_local(
        FAISS_DB_PATH, 
        embedding_model, 
        allow_dangerous_deserialization=True
    )

    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
    llm = Ollama(model="llama3.2:3b")
        
        # Define custom prompt using PromptTemplate
    prompt_template = PromptTemplate(
        input_variables=["information"],
        template="""
        Use the following piece of {information} to provide the proper KPI to use
        
        """
    )

    retrievalQA = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_template} 
    )

    return retrievalQA

In [ ]:
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain import hub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_community.llms import Ollama
from OprFuncs import data_infer, extract_code, extract_questions
import pandas as pd

# Initialize Ollama
llm = Ollama(model="llama3.2:3b")
docx_file_path = "kpis.docx"
if not os.path.exists(docx_file_path):
    raise FileNotFoundError(f"🚨 Error: The file '{docx_file_path}' was not found!")

documents = load_analysis_rules_from_memory(docx_file_path)
    
retrievalQA = train_rag_system(documents)
dataframe = pd.read_csv("Test_Datasets/supply_chain_data.csv")

tools = [

    Tool(
        name="understander",
        func=understander,  
        description="Provide usable information."
        )
    Tool(
        name="retrieval",
        func=retrieval,  
        description="Use this tool to provide a kpi based on the information."
        )
    Tool(
        name="analyzer",
        func=analysis_data,  
        description="Use this tool anlysis the dataframe using the kpi from the retrival tool."
        )
]
agent_prompt = hub.pull("hwchase17/react").partial(
    instructions="""Follow EXACTLY this sequence:
    1. Use understander ONCE
    2. Use retrieval ONCE
    3. Use analyzer ONCE
    NEVER repeat steps or tools"""
)
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True,
    stop=["\nFINAL ANSWER"] 
)

result = agent_executor.invoke({
    "input": f"""Analyze this question and generate visualization code:
    Question: {question}
    Follow this EXACT format:
    Thought: First analyze the question
    Action: ChartSelector
    Action Input: "{question}"
    Observation: [chart-type]
    Thought: Now generate code
    Action: CodeGenerator
    Action Input: "{question}|[chart-type]"
    FINAL ANSWER:"""
})

print(result["output"])

In [1]:
def understander(df,llm):
    data_info = data_infer(df)

    understanding_prompt = ''' 
    You are a data analyst. You are provided with a dataset about {data_info}.
    Here is the dataset structure:
    {data_info}

    Please understand what's the data about, you will provide 1 information in order to create the KPI, 
    return only the information about the data (what's data about, what column or relation you want to create the KPI for)

    '''

    understanding_chain = LLMChain(llm=llm, prompt=understanding_prompt)

        
    understanding = understanding_chain.run(data_info=data_info)

    return understanding

In [3]:
def data_describer(dataframe):
    # Get the description of the dataframe
    description = dataframe.describe()
    
    # Convert the description to a string with column names
    description_str = "Data Description:\n"
    for col in description.columns:
        description_str += f"\nColumn: {col}\n"
        description_str += description[col].to_string() + "\n"
    
    # Write the description to a file
    with open("df_description.txt", "w", encoding="utf-8") as f:
        f.write(description_str)
    
    return description_str

In [ ]:
data_sample = dataframe.head().to_string
data_summary = data_describer(dataframe)

In [ ]:
def analysis_data(df,llm):
    data_info = data_infer(df)
    data_sample = dataframe.head().to_string
    data_summary = data_describer(dataframe)

    analysis_prompt = '''
        You are a data analyst. You are provided with:
        1. Dataset metadata: {data_info}
        2. Dataset sample: {data_sample}
        3. Dataset summary: {data_summary} 
        4. KPIs:{kpi}

        Please analyze the data and provide insights about:
        Key trends and patterns using the {kpi}.
        '''
    analysis_template = PromptTemplate(
            input_variables=["data_info","data_sample","data_summary","kpi"],
            template=analysis_prompt
        )
        
    analysis_chain = LLMChain(llm=llm, prompt=analysis_template)

        
    analysis = analysis_chain.run(data_info=data_info,data_sample=data_sample,data_summary=data_summary,kpi=kpi)


    return analysis